# Create a MDAnalysis universe

In [1]:
import numpy as np
import MDAnalysis as mda
import nmrformd as nmrmd

In [2]:
datapath = "./lammps-inputs/"

In [3]:
u = mda.Universe(datapath+"topology.data", datapath+"traj.xtc")

In [4]:
n_molecules = u.atoms.n_residues
print(f"The number of water molecules is {n_molecules}")

The number of water molecules is 300


In [5]:
timestep = np.int32(u.trajectory.dt)
print(f"The timestep is {timestep} ps")

The timestep is 1 ps


In [6]:
total_time = np.int32(u.trajectory.totaltime)
print(f"The total simulation time is {total_time} ps")

The total simulation time is 1000 ps


# Run NMRforMD

In [9]:
group_i = u.select_atoms("type 2")

In [18]:
type(group_i) == mda.core.groups.AtomGroup

False

True

In [15]:
mda.core.groups.AtomGroup

MDAnalysis.core.groups.AtomGroup

In [10]:
nmr_result = nmrmd.NMR(u, group_i, number_i=40)

<AtomGroup [<Atom 2: of type 2 resid 1 and segid SYSTEM>, <Atom 3: of type 2 resid 1 and segid SYSTEM>, <Atom 5: of type 2 resid 2 and segid SYSTEM>, ..., <Atom 897: of type 2 resid 299 and segid SYSTEM>, <Atom 899: of type 2 resid 300 and segid SYSTEM>, <Atom 900: of type 2 resid 300 and segid SYSTEM>]>


AttributeError: AtomGroup has no attribute replace. 

In [9]:
nmr_result.atom_group

NameError: name 'nmr_result' is not defined

# Extract T1

In [ ]:
T1 = np.round(nmr_result.T1,2)
print(f"NMR relaxation time T1 = {T1} s")

# Plot the spectrum

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
fontsize = 30
font = {'family': 'sans', 'color':  'black', 'weight': 'normal', 'size': fontsize}
myblue = [0/ 255, 150/255, 177/ 255]
lightgray = [0.1, 0.1, 0.1]
darkgray = [0.9, 0.9, 0.9]

#### Plot one figure for the light mode, and one figure for the dark mode

In [ ]:
for mode, mygray in zip(['light', 'dark'], [lightgray, darkgray]):

    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    plt.loglog(nmr_result.f[:-60], 1/nmr_result.R1[:-60], 'o', markersize=8, color=myblue)
    divider = make_axes_locatable(ax1)
    ax1.set_xlabel(r"$f$ (MHz)", fontdict=font)
    ax1.set_ylabel(r'$T_1$ (s)', fontdict=font)
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    #plt.ylim(5, 65)
    ax1.minorticks_on()
    ax1.tick_params('both', length=10, width=2, which='major', direction='in')
    ax1.tick_params('both', length=6, width=1.4, which='minor', direction='in')
    ax1.xaxis.set_ticks_position('both')
    ax1.yaxis.set_ticks_position('both')
    ax1.spines["top"].set_linewidth(2)
    ax1.spines["bottom"].set_linewidth(2)
    ax1.spines["left"].set_linewidth(2)
    ax1.spines["right"].set_linewidth(2)
    #ax1.set_xticks([0, 5, 10, 15, 20])
    #ax1.set_yticks([0, 0.4, 0.8, 1.2, 1.6])
    #minor_locator_y = AutoMinorLocator(2)
    #ax1.yaxis.set_minor_locator(minor_locator_y)
    #minor_locator_x = AutoMinorLocator(2)
    #ax1.xaxis.set_minor_locator(minor_locator_x)
    ax1.xaxis.label.set_color(mygray)
    ax1.yaxis.label.set_color(mygray)
    ax1.tick_params(axis='x', colors=mygray)
    ax1.tick_params(axis='y', colors=mygray)
    ax1.spines['left'].set_color(mygray)
    ax1.spines['top'].set_color(mygray)
    ax1.spines['bottom'].set_color(mygray)
    ax1.spines['right'].set_color(mygray)
    ax1.tick_params(axis='y', which='both', colors=mygray)
    ax1.tick_params(axis='x', which='both', colors=mygray)
    ax1.tick_params(axis='x', pad=10)
    plt.xlim(5e2, 5e5)
    plt.ylim(1, 100)

    fig.tight_layout()
    if mode == 'light':
        plt.savefig('figures/T1-light.png', bbox_inches = 'tight', pad_inches = 0.057, transparent=True)
    else:
        plt.savefig('figures/T1-dark.png', bbox_inches = 'tight', pad_inches = 0.057, transparent=True)
    plt.show()

# Plot the correlation function

In [ ]:
for mode, mygray in zip(['light', 'dark'], [lightgray, darkgray]):

    fig = plt.figure(figsize=(16, 8))
    ax1 = fig.add_subplot(111)
    ax1.loglog(nmr_result.t[:-20], nmr_result.gij[0][:-20], 'o', markersize=8, color=myblue)
    divider = make_axes_locatable(ax1)
    ax1.set_xlabel(r"$t$ (ps)", fontdict=font)
    ax1.set_ylabel(r'$G$ (m$^{-6}$)', fontdict=font)
    plt.xticks(fontsize=fontsize)
    plt.yticks(fontsize=fontsize)
    #plt.ylim(5, 65)
    ax1.minorticks_on()
    ax1.tick_params('both', length=10, width=2, which='major', direction='in')
    ax1.tick_params('both', length=6, width=1.4, which='minor', direction='in')
    ax1.xaxis.set_ticks_position('both')
    ax1.yaxis.set_ticks_position('both')
    ax1.spines["top"].set_linewidth(2)
    ax1.spines["bottom"].set_linewidth(2)
    ax1.spines["left"].set_linewidth(2)
    ax1.spines["right"].set_linewidth(2)
    #ax1.set_xticks([0, 5, 10, 15, 20])
    #ax1.set_yticks([0, 0.4, 0.8, 1.2, 1.6])
    #minor_locator_y = AutoMinorLocator(2)
    #ax1.yaxis.set_minor_locator(minor_locator_y)
    #minor_locator_x = AutoMinorLocator(2)
    #ax1.xaxis.set_minor_locator(minor_locator_x)
    ax1.xaxis.label.set_color(mygray)
    ax1.yaxis.label.set_color(mygray)
    ax1.tick_params(axis='x', colors=mygray)
    ax1.tick_params(axis='y', colors=mygray)
    ax1.spines['left'].set_color(mygray)
    ax1.spines['top'].set_color(mygray)
    ax1.spines['bottom'].set_color(mygray)
    ax1.spines['right'].set_color(mygray)
    ax1.tick_params(axis='y', which='both', colors=mygray)
    ax1.tick_params(axis='x', which='both', colors=mygray)
    ax1.tick_params(axis='x', pad=10)
    #plt.xlim(5e2, 5e5)
    #plt.ylim(1, 100)
    fig.tight_layout()
    if mode == 'light':
        plt.savefig('figures/G-light.png', bbox_inches = 'tight', pad_inches = 0.057, transparent=True)
    else:
        plt.savefig('figures/G-dark.png', bbox_inches = 'tight', pad_inches = 0.057, transparent=True)
    plt.show()